In [1]:
import pandas as pd
import re
import jieba
import warnings
warnings.filterwarnings('ignore')

def clean_bilibili_data(input_path="D:/CITYU COURSES/5507/homework/评论/bilibili.xlsx", output_path="D:/CITYU COURSES/5507/homework/评论/cleaned_bilibili.xlsx"):
    """
    定制化清洗B站评论数据（适配上传的bilibili.xlsx文件）
    核心功能：去重、去噪、文本标准化、分词处理
    """
    # ---------------------- 1. 读取原始数据 ----------------------
    print("正在读取数据...")
    df = pd.read_excel(input_path, engine="openpyxl")
    original_count = len(df)
    print(f"原始数据：{original_count}条评论")

    # ---------------------- 2. 基础数据过滤 ----------------------
    # 2.1 删除完全重复的评论
    df = df.drop_duplicates(subset=["content"])
    # 2.2 删除评论内容为空/NaN的条目
    df = df.dropna(subset=["content"])
    # 2.3 过滤字数过短的无意义评论（少于2字）
    df = df[df["content"].str.len() >= 2]
    # 2.4 重置索引（避免清洗后索引不连续）
    df = df.reset_index(drop=True)
    basic_clean_count = len(df)
    print(f"基础过滤后：{basic_clean_count}条评论（去除重复/空值/短评论）")

    # ---------------------- 3. 深度文本清洗（去噪+标准化） ----------------------
    def clean_single_comment(text):
        """单条评论清洗：去除B站特有噪声，统一格式"""
        text = str(text).strip()
        
        # 3.1 去除B站特殊元素
        text = re.sub(r"\[tv_\w+\]|\[热词系列_\w+\]|\[\w+\]", "", text)  # 去除表情（如[tv_点赞]）
        text = re.sub(r"https?://\S+|b23.tv/\S+", "", text)              # 去除URL链接（含B站短链）
        text = re.sub(r"@\w+|#\w+#", "", text)                            # 去除@用户和#话题标签
        
        # 3.2 文本格式标准化
        text = re.sub(r"\n+", "。", text)  # 换行符转为句号（避免断句混乱）
        text = re.sub(r"\s+", " ", text)   # 多个空格合并为一个
        text = text.replace(",", "，").replace(".", "。").replace("?", "？").replace("!", "！")  # 统一中文标点
        
        # 3.3 保留有效字符（仅保留中文、英文、数字、常用中文标点）
        text = re.sub(r"[^\u4e00-\u9fa5a-zA-Z0-9\s，。！？；：“”‘’（）【】、]", "", text)
        
        return text.strip()

    # 应用清洗函数，新增“清洗后评论”字段
    df["cleaned_content"] = df["content"].apply(clean_single_comment)
    
    # 过滤清洗后为空/仍过短的评论（如原评论全是表情/链接）
    df = df[df["cleaned_content"].str.len() >= 2]
    df = df.reset_index(drop=True)
    deep_clean_count = len(df)
    print(f"深度清洗后：{deep_clean_count}条评论（去除噪声/标准化格式）")

    # ---------------------- 4. 分词处理（便于后续分析） ----------------------
    # 加载停用词表（过滤无意义词汇，保留核心语义）
    stopwords = {
        "的", "了", "是", "我", "你", "他", "她", "它", "们", "在", "和", "就", "都", "而", "及", "与",
        "也", "还", "不", "没", "有", "着", "过", "要", "会", "能", "可", "这", "那", "此", "彼",
        "很", "非常", "比较", "太", "最", "更", "又", "再", "才", "只", "但", "却", "虽", "然",
        "，", "。", "！", "？", "；", "：", "“", "”", "‘", "’", "（", "）", "【", "】", "、", " ",
        "回复", "引用", "举报", "删除", "编辑", "一个", "一些", "一点", "一样"
    }

    def segment_text(text):
        """中文分词并过滤停用词"""
        words = jieba.lcut(text, cut_all=False)  # 精确分词
        filtered_words = [
            word for word in words 
            if word not in stopwords 
            and len(word) >= 1 
            and not (word.isdigit() and len(word) < 5)  # 保留长度≥5的数字（如年份）
        ]
        return filtered_words

    # 新增分词相关字段
    df["segmented_words"] = df["cleaned_content"].apply(segment_text)  # 列表格式（便于程序处理）
    df["segmented_text"] = df["segmented_words"].apply(lambda x: " ".join(x))  # 字符串格式（便于查看）

    # ---------------------- 5. 保存清洗结果 ----------------------
    # 保留核心字段：原始评论、清洗后评论、分词结果（列表）、分词结果（字符串）
    core_columns = ["content", "cleaned_content", "segmented_words", "segmented_text"]
    df_cleaned = df[core_columns]
    df_cleaned.to_excel(output_path, index=False, engine="openpyxl")

    # ---------------------- 6. 输出清洗报告 ----------------------
    print(f"\n=== 清洗完成报告 ===")
    print(f"原始数据：{original_count}条")
    print(f"最终有效数据：{deep_clean_count}条")
    print(f"数据保留率：{deep_clean_count/original_count*100:.1f}%")
    print(f"清洗后文件路径：{output_path}")
    
    # 展示前2条清洗效果对比
    print(f"\n=== 清洗效果示例 ===")
    for i in range(min(2, len(df_cleaned))):
        print(f"\n【原始评论{i+1}】：")
        original = df_cleaned.iloc[i]["content"]
        print(f"  {original[:100]}{'...' if len(original) > 100 else ''}")
        print(f"【清洗后评论{i+1}】：")
        cleaned = df_cleaned.iloc[i]["cleaned_content"]
        print(f"  {cleaned[:100]}{'...' if len(cleaned) > 100 else ''}")
        print(f"【分词结果{i+1}】：")
        segmented = df_cleaned.iloc[i]["segmented_text"]
        print(f"  {segmented[:50]}{'...' if len(segmented) > 50 else ''}")

    return df_cleaned

# ---------------------- 直接运行清洗 ----------------------
if __name__ == "__main__":
    cleaned_data = clean_bilibili_data()

正在读取数据...


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\82588\AppData\Local\Temp\jieba.cache


原始数据：3142条评论
基础过滤后：3123条评论（去除重复/空值/短评论）
深度清洗后：3109条评论（去除噪声/标准化格式）


Loading model cost 0.826 seconds.
Prefix dict has been built successfully.



=== 清洗完成报告 ===
原始数据：3142条
最终有效数据：3109条
数据保留率：98.9%
清洗后文件路径：D:/CITYU COURSES/5507/homework/评论/cleaned_bilibili.xlsx

=== 清洗效果示例 ===

【原始评论1】：
  我寻思木乃伊也是有的啊
【清洗后评论1】：
  我寻思木乃伊也是有的啊
【分词结果1】：
  寻思 木乃伊 啊

【原始评论2】：
  做标本需要太多的时间，太多的流程！需要理解一下他们！[热词系列_保护]
【清洗后评论2】：
  做标本需要太多的时间，太多的流程！需要理解一下他们！
【分词结果2】：
  做 标本 需要 太多 时间 太多 流程 需要 理解 一下 他们


In [1]:
import pandas as pd
import re
import jieba
import warnings
warnings.filterwarnings('ignore')

def clean_bilibili_data(input_path="D:/CITYU COURSES/5507/homework/评论/comment4.xlsx", output_path="D:/CITYU COURSES/5507/homework/评论/cleaned_bilibili.xlsx"):
    """
    定制化清洗B站评论数据（适配上传的bilibili.xlsx文件）
    核心功能：去重、去噪、文本标准化、分词处理
    """
    # ---------------------- 1. 读取原始数据 ----------------------
    print("正在读取数据...")
    df = pd.read_excel(input_path, engine="openpyxl")
    original_count = len(df)
    print(f"原始数据：{original_count}条评论")

    # ---------------------- 2. 基础数据过滤 ----------------------
    # 2.1 删除完全重复的评论
    df = df.drop_duplicates(subset=["content"])
    # 2.2 删除评论内容为空/NaN的条目
    df = df.dropna(subset=["content"])
    # 2.3 过滤字数过短的无意义评论（少于2字）
    df = df[df["content"].str.len() >= 2]
    # 2.4 重置索引（避免清洗后索引不连续）
    df = df.reset_index(drop=True)
    basic_clean_count = len(df)
    print(f"基础过滤后：{basic_clean_count}条评论（去除重复/空值/短评论）")

    # ---------------------- 3. 深度文本清洗（去噪+标准化） ----------------------
    def clean_single_comment(text):
        """单条评论清洗：去除B站特有噪声，统一格式"""
        text = str(text).strip()
        
        # 3.1 去除B站特殊元素
        text = re.sub(r"\[tv_\w+\]|\[热词系列_\w+\]|\[\w+\]", "", text)  # 去除表情（如[tv_点赞]）
        text = re.sub(r"https?://\S+|b23.tv/\S+", "", text)              # 去除URL链接（含B站短链）
        text = re.sub(r"@\w+|#\w+#", "", text)                            # 去除@用户和#话题标签
        
        # 3.2 文本格式标准化
        text = re.sub(r"\n+", "。", text)  # 换行符转为句号（避免断句混乱）
        text = re.sub(r"\s+", " ", text)   # 多个空格合并为一个
        text = text.replace(",", "，").replace(".", "。").replace("?", "？").replace("!", "！")  # 统一中文标点
        
        # 3.3 保留有效字符（仅保留中文、英文、数字、常用中文标点）
        text = re.sub(r"[^\u4e00-\u9fa5a-zA-Z0-9\s，。！？；：“”‘’（）【】、]", "", text)
        
        return text.strip()

    # 应用清洗函数，新增“清洗后评论”字段
    df["cleaned_content"] = df["content"].apply(clean_single_comment)
    
    # 过滤清洗后为空/仍过短的评论（如原评论全是表情/链接）
    df = df[df["cleaned_content"].str.len() >= 2]
    df = df.reset_index(drop=True)
    deep_clean_count = len(df)
    print(f"深度清洗后：{deep_clean_count}条评论（去除噪声/标准化格式）")

    # ---------------------- 4. 分词处理（便于后续分析） ----------------------
    # 加载停用词表（过滤无意义词汇，保留核心语义）
    stopwords = {
        "的", "了", "是", "我", "你", "他", "她", "它", "们", "在", "和", "就", "都", "而", "及", "与",
        "也", "还", "不", "没", "有", "着", "过", "要", "会", "能", "可", "这", "那", "此", "彼",
        "很", "非常", "比较", "太", "最", "更", "又", "再", "才", "只", "但", "却", "虽", "然",
        "，", "。", "！", "？", "；", "：", "“", "”", "‘", "’", "（", "）", "【", "】", "、", " ",
        "回复", "引用", "举报", "删除", "编辑", "一个", "一些", "一点", "一样"
    }

    def segment_text(text):
        """中文分词并过滤停用词"""
        words = jieba.lcut(text, cut_all=False)  # 精确分词
        filtered_words = [
            word for word in words 
            if word not in stopwords 
            and len(word) >= 1 
            and not (word.isdigit() and len(word) < 5)  # 保留长度≥5的数字（如年份）
        ]
        return filtered_words

    # 新增分词相关字段
    df["segmented_words"] = df["cleaned_content"].apply(segment_text)  # 列表格式（便于程序处理）
    df["segmented_text"] = df["segmented_words"].apply(lambda x: " ".join(x))  # 字符串格式（便于查看）

    # ---------------------- 5. 保存清洗结果 ----------------------
    # 保留核心字段：原始评论、清洗后评论、分词结果（列表）、分词结果（字符串）
    core_columns = ["content", "cleaned_content", "segmented_words", "segmented_text"]
    df_cleaned = df[core_columns]
    df_cleaned.to_excel(output_path, index=False, engine="openpyxl")

    # ---------------------- 6. 输出清洗报告 ----------------------
    print(f"\n=== 清洗完成报告 ===")
    print(f"原始数据：{original_count}条")
    print(f"最终有效数据：{deep_clean_count}条")
    print(f"数据保留率：{deep_clean_count/original_count*100:.1f}%")
    print(f"清洗后文件路径：{output_path}")
    
    # 展示前2条清洗效果对比
    print(f"\n=== 清洗效果示例 ===")
    for i in range(min(2, len(df_cleaned))):
        print(f"\n【原始评论{i+1}】：")
        original = df_cleaned.iloc[i]["content"]
        print(f"  {original[:100]}{'...' if len(original) > 100 else ''}")
        print(f"【清洗后评论{i+1}】：")
        cleaned = df_cleaned.iloc[i]["cleaned_content"]
        print(f"  {cleaned[:100]}{'...' if len(cleaned) > 100 else ''}")
        print(f"【分词结果{i+1}】：")
        segmented = df_cleaned.iloc[i]["segmented_text"]
        print(f"  {segmented[:50]}{'...' if len(segmented) > 50 else ''}")

    return df_cleaned

# ---------------------- 直接运行清洗 ----------------------
if __name__ == "__main__":
    cleaned_data = clean_bilibili_data()

正在读取数据...


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\82588\AppData\Local\Temp\jieba.cache


原始数据：691条评论
基础过滤后：676条评论（去除重复/空值/短评论）
深度清洗后：663条评论（去除噪声/标准化格式）


Loading model cost 0.671 seconds.
Prefix dict has been built successfully.



=== 清洗完成报告 ===
原始数据：691条
最终有效数据：663条
数据保留率：95.9%
清洗后文件路径：D:/CITYU COURSES/5507/homework/评论/cleaned_bilibili.xlsx

=== 清洗效果示例 ===

【原始评论1】：
  。嚎啕大哭
【清洗后评论1】：
  。嚎啕大哭
【分词结果1】：
  嚎啕大哭

【原始评论2】：
  看哭了😭家里也有一只毛孩子，未来得有多舍不得
【清洗后评论2】：
  看哭了家里也有一只毛孩子，未来得有多舍不得
【分词结果2】：
  看 哭 家里 一只 毛孩子 未来 得 多 舍不得


In [1]:
import pandas as pd
import re
import jieba
import warnings
warnings.filterwarnings('ignore')

def clean_bilibili_data(input_path="D:/CITYU COURSES/5507/homework/评论/comment5.xlsx", output_path="D:/CITYU COURSES/5507/homework/评论/cleaned_bilibili.xlsx"):
    """
    定制化清洗B站评论数据（适配上传的bilibili.xlsx文件）
    核心功能：去重、去噪、文本标准化、分词处理
    """
    # ---------------------- 1. 读取原始数据 ----------------------
    print("正在读取数据...")
    df = pd.read_excel(input_path, engine="openpyxl")
    original_count = len(df)
    print(f"原始数据：{original_count}条评论")

    # ---------------------- 2. 基础数据过滤 ----------------------
    # 2.1 删除完全重复的评论
    df = df.drop_duplicates(subset=["content"])
    # 2.2 删除评论内容为空/NaN的条目
    df = df.dropna(subset=["content"])
    # 2.3 过滤字数过短的无意义评论（少于2字）
    df = df[df["content"].str.len() >= 2]
    # 2.4 重置索引（避免清洗后索引不连续）
    df = df.reset_index(drop=True)
    basic_clean_count = len(df)
    print(f"基础过滤后：{basic_clean_count}条评论（去除重复/空值/短评论）")

    # ---------------------- 3. 深度文本清洗（去噪+标准化） ----------------------
    def clean_single_comment(text):
        """单条评论清洗：去除B站特有噪声，统一格式"""
        text = str(text).strip()
        
        # 3.1 去除B站特殊元素
        text = re.sub(r"\[tv_\w+\]|\[热词系列_\w+\]|\[\w+\]", "", text)  # 去除表情（如[tv_点赞]）
        text = re.sub(r"https?://\S+|b23.tv/\S+", "", text)              # 去除URL链接（含B站短链）
        text = re.sub(r"@\w+|#\w+#", "", text)                            # 去除@用户和#话题标签
        
        # 3.2 文本格式标准化
        text = re.sub(r"\n+", "。", text)  # 换行符转为句号（避免断句混乱）
        text = re.sub(r"\s+", " ", text)   # 多个空格合并为一个
        text = text.replace(",", "，").replace(".", "。").replace("?", "？").replace("!", "！")  # 统一中文标点
        
        # 3.3 保留有效字符（仅保留中文、英文、数字、常用中文标点）
        text = re.sub(r"[^\u4e00-\u9fa5a-zA-Z0-9\s，。！？；：“”‘’（）【】、]", "", text)
        
        return text.strip()

    # 应用清洗函数，新增“清洗后评论”字段
    df["cleaned_content"] = df["content"].apply(clean_single_comment)
    
    # 过滤清洗后为空/仍过短的评论（如原评论全是表情/链接）
    df = df[df["cleaned_content"].str.len() >= 2]
    df = df.reset_index(drop=True)
    deep_clean_count = len(df)
    print(f"深度清洗后：{deep_clean_count}条评论（去除噪声/标准化格式）")

    # ---------------------- 4. 分词处理（便于后续分析） ----------------------
    # 加载停用词表（过滤无意义词汇，保留核心语义）
    stopwords = {
        "的", "了", "是", "我", "你", "他", "她", "它", "们", "在", "和", "就", "都", "而", "及", "与",
        "也", "还", "不", "没", "有", "着", "过", "要", "会", "能", "可", "这", "那", "此", "彼",
        "很", "非常", "比较", "太", "最", "更", "又", "再", "才", "只", "但", "却", "虽", "然",
        "，", "。", "！", "？", "；", "：", "“", "”", "‘", "’", "（", "）", "【", "】", "、", " ",
        "回复", "引用", "举报", "删除", "编辑", "一个", "一些", "一点", "一样"
    }

    def segment_text(text):
        """中文分词并过滤停用词"""
        words = jieba.lcut(text, cut_all=False)  # 精确分词
        filtered_words = [
            word for word in words 
            if word not in stopwords 
            and len(word) >= 1 
            and not (word.isdigit() and len(word) < 5)  # 保留长度≥5的数字（如年份）
        ]
        return filtered_words

    # 新增分词相关字段
    df["segmented_words"] = df["cleaned_content"].apply(segment_text)  # 列表格式（便于程序处理）
    df["segmented_text"] = df["segmented_words"].apply(lambda x: " ".join(x))  # 字符串格式（便于查看）

    # ---------------------- 5. 保存清洗结果 ----------------------
    # 保留核心字段：原始评论、清洗后评论、分词结果（列表）、分词结果（字符串）
    core_columns = ["content", "cleaned_content", "segmented_words", "segmented_text"]
    df_cleaned = df[core_columns]
    df_cleaned.to_excel(output_path, index=False, engine="openpyxl")

    # ---------------------- 6. 输出清洗报告 ----------------------
    print(f"\n=== 清洗完成报告 ===")
    print(f"原始数据：{original_count}条")
    print(f"最终有效数据：{deep_clean_count}条")
    print(f"数据保留率：{deep_clean_count/original_count*100:.1f}%")
    print(f"清洗后文件路径：{output_path}")
    
    # 展示前2条清洗效果对比
    print(f"\n=== 清洗效果示例 ===")
    for i in range(min(2, len(df_cleaned))):
        print(f"\n【原始评论{i+1}】：")
        original = df_cleaned.iloc[i]["content"]
        print(f"  {original[:100]}{'...' if len(original) > 100 else ''}")
        print(f"【清洗后评论{i+1}】：")
        cleaned = df_cleaned.iloc[i]["cleaned_content"]
        print(f"  {cleaned[:100]}{'...' if len(cleaned) > 100 else ''}")
        print(f"【分词结果{i+1}】：")
        segmented = df_cleaned.iloc[i]["segmented_text"]
        print(f"  {segmented[:50]}{'...' if len(segmented) > 50 else ''}")

    return df_cleaned

# ---------------------- 直接运行清洗 ----------------------
if __name__ == "__main__":
    cleaned_data = clean_bilibili_data()

正在读取数据...


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\82588\AppData\Local\Temp\jieba.cache


原始数据：179条评论
基础过滤后：179条评论（去除重复/空值/短评论）
深度清洗后：176条评论（去除噪声/标准化格式）


Loading model cost 0.788 seconds.
Prefix dict has been built successfully.



=== 清洗完成报告 ===
原始数据：179条
最终有效数据：176条
数据保留率：98.3%
清洗后文件路径：D:/CITYU COURSES/5507/homework/评论/cleaned_bilibili.xlsx

=== 清洗效果示例 ===

【原始评论1】：
  我的第一反应就是！ 这还是一片蓝海啊！[微笑]
【清洗后评论1】：
  我的第一反应就是！ 这还是一片蓝海啊！
【分词结果1】：
  第一 反应 就是 还是 一片 蓝海 啊

【原始评论2】：
  每个生命都可贵
【清洗后评论2】：
  每个生命都可贵
【分词结果2】：
  每个 生命 可贵


In [1]:
import pandas as pd
import re
import jieba
import warnings
warnings.filterwarnings('ignore')

def clean_bilibili_data(input_path="D:/CITYU COURSES/5507/homework/评论/comment6.xlsx", output_path="D:/CITYU COURSES/5507/homework/评论/cleaned_bilibili.xlsx"):
    """
    定制化清洗B站评论数据（适配上传的bilibili.xlsx文件）
    核心功能：去重、去噪、文本标准化、分词处理
    """
    # ---------------------- 1. 读取原始数据 ----------------------
    print("正在读取数据...")
    df = pd.read_excel(input_path, engine="openpyxl")
    original_count = len(df)
    print(f"原始数据：{original_count}条评论")

    # ---------------------- 2. 基础数据过滤 ----------------------
    # 2.1 删除完全重复的评论
    df = df.drop_duplicates(subset=["content"])
    # 2.2 删除评论内容为空/NaN的条目
    df = df.dropna(subset=["content"])
    # 2.3 过滤字数过短的无意义评论（少于2字）
    df = df[df["content"].str.len() >= 2]
    # 2.4 重置索引（避免清洗后索引不连续）
    df = df.reset_index(drop=True)
    basic_clean_count = len(df)
    print(f"基础过滤后：{basic_clean_count}条评论（去除重复/空值/短评论）")

    # ---------------------- 3. 深度文本清洗（去噪+标准化） ----------------------
    def clean_single_comment(text):
        """单条评论清洗：去除B站特有噪声，统一格式"""
        text = str(text).strip()
        
        # 3.1 去除B站特殊元素
        text = re.sub(r"\[tv_\w+\]|\[热词系列_\w+\]|\[\w+\]", "", text)  # 去除表情（如[tv_点赞]）
        text = re.sub(r"https?://\S+|b23.tv/\S+", "", text)              # 去除URL链接（含B站短链）
        text = re.sub(r"@\w+|#\w+#", "", text)                            # 去除@用户和#话题标签
        
        # 3.2 文本格式标准化
        text = re.sub(r"\n+", "。", text)  # 换行符转为句号（避免断句混乱）
        text = re.sub(r"\s+", " ", text)   # 多个空格合并为一个
        text = text.replace(",", "，").replace(".", "。").replace("?", "？").replace("!", "！")  # 统一中文标点
        
        # 3.3 保留有效字符（仅保留中文、英文、数字、常用中文标点）
        text = re.sub(r"[^\u4e00-\u9fa5a-zA-Z0-9\s，。！？；：“”‘’（）【】、]", "", text)
        
        return text.strip()

    # 应用清洗函数，新增“清洗后评论”字段
    df["cleaned_content"] = df["content"].apply(clean_single_comment)
    
    # 过滤清洗后为空/仍过短的评论（如原评论全是表情/链接）
    df = df[df["cleaned_content"].str.len() >= 2]
    df = df.reset_index(drop=True)
    deep_clean_count = len(df)
    print(f"深度清洗后：{deep_clean_count}条评论（去除噪声/标准化格式）")

    # ---------------------- 4. 分词处理（便于后续分析） ----------------------
    # 加载停用词表（过滤无意义词汇，保留核心语义）
    stopwords = {
        "的", "了", "是", "我", "你", "他", "她", "它", "们", "在", "和", "就", "都", "而", "及", "与",
        "也", "还", "不", "没", "有", "着", "过", "要", "会", "能", "可", "这", "那", "此", "彼",
        "很", "非常", "比较", "太", "最", "更", "又", "再", "才", "只", "但", "却", "虽", "然",
        "，", "。", "！", "？", "；", "：", "“", "”", "‘", "’", "（", "）", "【", "】", "、", " ",
        "回复", "引用", "举报", "删除", "编辑", "一个", "一些", "一点", "一样"
    }

    def segment_text(text):
        """中文分词并过滤停用词"""
        words = jieba.lcut(text, cut_all=False)  # 精确分词
        filtered_words = [
            word for word in words 
            if word not in stopwords 
            and len(word) >= 1 
            and not (word.isdigit() and len(word) < 5)  # 保留长度≥5的数字（如年份）
        ]
        return filtered_words

    # 新增分词相关字段
    df["segmented_words"] = df["cleaned_content"].apply(segment_text)  # 列表格式（便于程序处理）
    df["segmented_text"] = df["segmented_words"].apply(lambda x: " ".join(x))  # 字符串格式（便于查看）

    # ---------------------- 5. 保存清洗结果 ----------------------
    # 保留核心字段：原始评论、清洗后评论、分词结果（列表）、分词结果（字符串）
    core_columns = ["content", "cleaned_content", "segmented_words", "segmented_text"]
    df_cleaned = df[core_columns]
    df_cleaned.to_excel(output_path, index=False, engine="openpyxl")

    # ---------------------- 6. 输出清洗报告 ----------------------
    print(f"\n=== 清洗完成报告 ===")
    print(f"原始数据：{original_count}条")
    print(f"最终有效数据：{deep_clean_count}条")
    print(f"数据保留率：{deep_clean_count/original_count*100:.1f}%")
    print(f"清洗后文件路径：{output_path}")
    
    # 展示前2条清洗效果对比
    print(f"\n=== 清洗效果示例 ===")
    for i in range(min(2, len(df_cleaned))):
        print(f"\n【原始评论{i+1}】：")
        original = df_cleaned.iloc[i]["content"]
        print(f"  {original[:100]}{'...' if len(original) > 100 else ''}")
        print(f"【清洗后评论{i+1}】：")
        cleaned = df_cleaned.iloc[i]["cleaned_content"]
        print(f"  {cleaned[:100]}{'...' if len(cleaned) > 100 else ''}")
        print(f"【分词结果{i+1}】：")
        segmented = df_cleaned.iloc[i]["segmented_text"]
        print(f"  {segmented[:50]}{'...' if len(segmented) > 50 else ''}")

    return df_cleaned

# ---------------------- 直接运行清洗 ----------------------
if __name__ == "__main__":
    cleaned_data = clean_bilibili_data()

正在读取数据...


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\82588\AppData\Local\Temp\jieba.cache


原始数据：1087条评论
基础过滤后：1075条评论（去除重复/空值/短评论）
深度清洗后：1059条评论（去除噪声/标准化格式）


Loading model cost 0.838 seconds.
Prefix dict has been built successfully.



=== 清洗完成报告 ===
原始数据：1087条
最终有效数据：1059条
数据保留率：97.4%
清洗后文件路径：D:/CITYU COURSES/5507/homework/评论/cleaned_bilibili.xlsx

=== 清洗效果示例 ===

【原始评论1】：
  第一[星星眼]
【清洗后评论1】：
  第一
【分词结果1】：
  第一

【原始评论2】：
  前十
【清洗后评论2】：
  前十
【分词结果2】：
  前十
